# CheXpert : A Large Chest X-Ray Dataset and Competition

This competition launched by the Stanford ML group aims at finding a prediction model which could perform as well as radiologist to find different pathologies thanks to chest X-Ray. The Dataset available to train our model is composed of 223,414 chest radiographs of 65,240 patients.

<img src="view1_frontal.jpg" title="X-Ray image of the dataset" width = 320/>

The website of the competition:
https://stanfordmlgroup.github.io/competitions/chexpert/

[Publication](https://arxiv.org/pdf/1901.07031.pdf) : Irvin, Jeremy, et al. "CheXpert: A Large Chest Radiograph Dataset with Uncertainty Labels and Expert Comparison." arXiv preprint arXiv:1901.07031 (2019).

Our goal is first to reproduce main results obtained in the related paper, published in January 2019.

In [1]:
import os
import numpy as np
import time
import sys
import csv
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as tfunc
from torch.utils.data import Dataset
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from PIL import Image
import torch.nn.functional as func

from sklearn.metrics.ranking import roc_auc_score

use_gpu = torch.cuda.is_available()

In [2]:
# Paths to the files with training, and validation sets.
# Each file contains pairs (path to image, output vector)
pathFileTrain = '../CheXpert-v1.0-small/train.csv'
pathFileValid = '../CheXpert-v1.0-small/valid.csv'

# Neural network parameters:
nnIsTrained = False                 #pre-trained using ImageNet
nnClassCount = 14                   #dimension of the output

# Training settings: batch size, maximum number of epochs
trBatchSize = 64
trMaxEpoch = 1

# Parameters related to image transforms: size of the down-scaled image, cropped image
imgtransResize = (320, 320)
imgtransCrop = 224

class_names = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 
       'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

## 1. Create a Dataset

In this first part, we create a loader able to load the data thanks to the csv files of the dataset.
This loader first load labels and paths to the different images.

It includes a pre-processing of labels to deal with uncertainty labels.
Indeed, four different labels exist:
-  **Blanks ('')**: the existence of the pathology is unknown
-  **Ones (1)**: the pathology is detected thanks to the image
-  **Zeros (0)**: the pathology can't be detected thanks to the image
-  **Uncertain (-1)**: the pathology may be detected.

In order to deal with uncertainty labels, different policies can be applied. The easiest ones consists in changing those **Uncertain** labels to *Blanks*, *Ones* or *Zeros*. This process is done thanks to the following class.

**PROBLEME BLANKS AVEC DATALOADER PYTORCH**

In [3]:
class CheXpertDataSet(Dataset):
    def __init__(self, image_list_file, transform=None, policy="ones", nmax = None):
        """
        Args:
            image_list_file: path to the file containing images
                with corresponding labels.
            transform: optional transform to be applied on a sample.
            Upolicy: name the policy with regard to the uncertain labels
        """
        image_names = []
        labels = []

        with open(image_list_file, "r") as f:
            csvReader = csv.reader(f)
            next(csvReader, None)
            k=0
            for line in csvReader:
                k+=1
                if nmax!=None and k>nmax:
                    break
                image_name= line[0]
                label = line[5:]
                
                for i in range(14):
                    if label[i]:
                        a = float(label[i])
                        if a == 1:
                            label[i] = 1
                        elif a == -1:
                            if policy == "ones":
                                label[i] = 1
                            elif policy == "zeroes":
                                label[i] = 0
                            else:
                                label[i] = 0
                        else:
                            label[i] = 0
                    else:
                        label[i] = 0
                        
                image_names.append('../' + image_name)
                labels.append(label)

        self.image_names = image_names
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        """
        Args:
            index: the index of item

        Returns:
            image and its labels
        """
        image_name = self.image_names[index]
        image = Image.open(image_name).convert('RGB')
        label = self.labels[index]
        if self.transform is not None:
            image = self.transform(image)
        return image, torch.FloatTensor(label)

    def __len__(self):
        return len(self.image_names)

## 2. Create DataLoaders

First we define a transform model to resize all images and normalize them.

In [4]:
#TRANSFORM DATA

normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
transformList = []
#transformList.append(transforms.Resize(imgtransResize)) #a regarder
transformList.append(transforms.RandomResizedCrop(imgtransCrop))
transformList.append(transforms.RandomHorizontalFlip())
transformList.append(transforms.ToTensor())
transformList.append(normalize)      
transformSequence=transforms.Compose(transformList)

Then we build train, validation and test data loaders. 

In [5]:
#LOAD DATASET

dataset = CheXpertDataSet(pathFileTrain ,transformSequence, policy=1)
datasetTest, datasetTrain = random_split(dataset, [500, len(dataset) - 500])
datasetValid = CheXpertDataSet(pathFileValid, transformSequence)            
#Problèmes de l'overlapping de patients et du transform identique ?

dataLoaderTrain = DataLoader(dataset=datasetTrain, batch_size=trBatchSize, shuffle=True,  num_workers=24, pin_memory=True)
dataLoaderVal = DataLoader(dataset=datasetValid, batch_size=trBatchSize, shuffle=False, num_workers=24, pin_memory=True)
dataLoaderTest = DataLoader(dataset=datasetTest, num_workers=24, pin_memory=True)

## 3. Create train and test models

First we define a transform model to resize all images and normalize them.

In [6]:
class CheXpertTrainer():

    #---- nnClassCount - number of output classes 
    #---- trMaxEpoch - number of epochs
    #---- launchTimestamp - date/time, used to assign unique name for the checkpoint file
    #---- checkpoint - if not None loads the model and continues training
    
    def train (model, dataLoaderTrain, dataLoaderVal, nnClassCount, trMaxEpoch, launchTimestamp, checkpoint):
        
        
        #SETTINGS: OPTIMIZER & SCHEDULER
        optimizer = optim.Adam (model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-5)
                
        #SETTINGS: LOSS
        loss = torch.nn.BCELoss(size_average = True)
        
        #LOAD CHECKPOINT 
        if checkpoint != None and use_gpu:
            modelCheckpoint = torch.load(checkpoint)
            model.load_state_dict(modelCheckpoint['state_dict'])
            optimizer.load_state_dict(modelCheckpoint['optimizer'])

        
        #TRAIN THE NETWORK
        
        lossMIN = 100000
        
        for epochID in range(0, trMaxEpoch):
            
            timestampTime = time.strftime("%H%M%S")
            timestampDate = time.strftime("%d%m%Y")
            timestampSTART = timestampDate + '-' + timestampTime
            
            batchs, losst, losse = CheXpertTrainer.epochTrain(model, dataLoaderTrain, optimizer, trMaxEpoch, nnClassCount, loss)
            lossVal = CheXpertTrainer.epochVal(model, dataLoaderVal, optimizer, trMaxEpoch, nnClassCount, loss)


            timestampTime = time.strftime("%H%M%S")
            timestampDate = time.strftime("%d%m%Y")
            timestampEND = timestampDate + '-' + timestampTime
            
            if lossVal < lossMIN:
                lossMIN = lossVal    
                torch.save({'epoch': epochID + 1, 'state_dict': model.state_dict(), 'best_loss': lossMIN, 'optimizer' : optimizer.state_dict()}, 'm-epoch'+str(epochID)+'-' + launchTimestamp + '.pth.tar')
                print ('Epoch [' + str(epochID + 1) + '] [save] [' + timestampEND + '] loss= ' + str(lossVal))
            else:
                print ('Epoch [' + str(epochID + 1) + '] [----] [' + timestampEND + '] loss= ' + str(lossVal))
        
        return batchs, losst, losse        
    #-------------------------------------------------------------------------------- 
       
    def epochTrain(model, dataLoader, optimizer, epochMax, classCount, loss):
        
        batch = []
        losstrain = []
        losseval = []
        
        model.train()

        for batchID, (varInput, target) in enumerate(dataLoaderTrain):
            
            varTarget = target.cuda(non_blocking = True) 
            varInput.requires_grad = True
            varOutput = model(varInput)
        
            lossvalue = loss(varOutput, varTarget)
                       
            optimizer.zero_grad()
            lossvalue.backward()
            optimizer.step()
            
            l = lossvalue.item()
            losstrain.append(l)
            
            if batchID%35==0:
                print(batchID//35, "% batches computed")
                #Fill three arrays to see the evolution of the loss


                batch.append(batchID)
                
                le = CheXpertTrainer.epochVal(model, dataLoaderVal, optimizer, trMaxEpoch, nnClassCount, loss).item()
                losseval.append(le)
                
                print(batchID)
                print(l)
                print(le)
                
        return batch, losstrain, losseval
    
    #-------------------------------------------------------------------------------- 
    
    def epochVal(model, dataLoader, optimizer, epochMax, classCount, loss):
        
        model.eval()
        
        lossVal = 0
        lossValNorm = 0

        with torch.no_grad():
            for i, (varInput, target) in enumerate(dataLoaderVal):
                
                target = target.cuda(non_blocking = True)
                varOutput = model(varInput)
                
                losstensor = loss(varOutput, target)
                lossVal += losstensor
                lossValNorm += 1
                
        outLoss = lossVal / lossValNorm
        return outLoss
    
    
    #--------------------------------------------------------------------------------     
     
    #---- Computes area under ROC curve 
    #---- dataGT - ground truth data
    #---- dataPRED - predicted data
    #---- classCount - number of classes
    
    def computeAUROC (dataGT, dataPRED, classCount):
        
        outAUROC = []
        
        datanpGT = dataGT.cpu().numpy()
        datanpPRED = dataPRED.cpu().numpy()
        print(datanpGT)
        print(datanpPRED)
        
        for i in range(classCount):
            outAUROC.append(roc_auc_score(datanpGT[:, i], datanpPRED[:, i]))
            
        return outAUROC
        
        
    #-------------------------------------------------------------------------------- 
    
    
    def test(model, dataLoaderTest, nnClassCount, checkpoint):   
        
        cudnn.benchmark = True
        
        if checkpoint != None and use_gpu:
            modelCheckpoint = torch.load(checkpoint)
            model.load_state_dict(modelCheckpoint['state_dict'])

        if use_gpu:
            outGT = torch.FloatTensor().cuda()
            outPRED = torch.FloatTensor().cuda()
        else:
            outGT = torch.FloatTensor()
            outPRED = torch.FloatTensor()
       
        model.eval()
        
        with torch.no_grad():
            for i, (input, target) in enumerate(dataLoaderTest):
                
                target = target.cuda()
                outGT = torch.cat((outGT, target), 0).cuda()

                bs, c, h, w = input.size()
                varInput = input.view(-1, c, h, w)
            
                out = model(varInput)
                print(out)

                outPRED = torch.cat((outPRED, out), 0)
                print(outPRED)
        aurocIndividual = CheXpertTrainer.computeAUROC(outGT, outPRED, nnClassCount)
        aurocMean = np.array(aurocIndividual).mean()
        
        print ('AUROC mean ', aurocMean)
        
        for i in range (0, len(aurocIndividual)):
            print (class_names[i], ' ', aurocIndividual[i])

In [8]:
class DenseNet121(nn.Module):
    """Model modified.

    The architecture of our model is the same as standard DenseNet121
    except the classifier layer which has an additional sigmoid function.

    """
    def __init__(self, out_size):
        super(DenseNet121, self).__init__()
        self.densenet121 = torchvision.models.densenet121(pretrained=True)
        num_ftrs = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Sequential(
            nn.Linear(num_ftrs, out_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.densenet121(x)
        return x

In [9]:
# initialize and load the model
if use_gpu:
    model = DenseNet121(nnClassCount).cuda()
    model = torch.nn.DataParallel(model).cuda()
else:
    model = DenseNet121(nnClassCount)
    model = torch.nn.DataParallel(model)

/home/gaetandi/miniconda3/lib/python3.7/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [ ]:
timestampTime = time.strftime("%H%M%S")
timestampDate = time.strftime("%d%m%Y")
timestampLaunch = timestampDate + '-' + timestampTime

#last model saved : "m-epoch0-07032019-213933.pth.tar"
batch, losst, losse = CheXpertTrainer.train(model, dataLoaderTrain, dataLoaderVal, nnClassCount, trMaxEpoch, timestampLaunch, checkpoint = "model1.pth.tar")
print("Model trained")

In [ ]:
losstn = []
for i in range(0, len(losst), 35):
    losstn.append(np.mean(losst[i:i+35])

In [9]:
losstn=[0.6728807091712952, 0.43882980942726135, 0.448734313249588, 0.3944702744483948, 0.4168243706226349, 0.4108811020851135, 0.4063291847705841, 0.41022342443466187, 0.3835395276546478, 0.3770408034324646, 0.43638280034065247, 0.3838731348514557, 0.41709738969802856, 0.3788347840309143, 0.4156818389892578, 0.40620699524879456, 0.39896196126937866, 0.41358619928359985, 0.3962070047855377, 0.3510574698448181, 0.4177820384502411, 0.38106709718704224, 0.39271387457847595, 0.39069268107414246, 0.4015863239765167, 0.4040599763393402, 0.36674314737319946, 0.39783015847206116, 0.39709019660949707, 0.39561790227890015, 0.3752536475658417, 0.3812060058116913, 0.3846367299556732, 0.37263208627700806, 0.39953988790512085, 0.38198143243789673, 0.38135018944740295, 0.3660784065723419, 0.3849000930786133, 0.39580315351486206, 0.37385669350624084, 0.4011920392513275, 0.38841670751571655, 0.39842209219932556, 0.37201929092407227, 0.3926997184753418, 0.3799830377101898, 0.34905099868774414, 0.39441245794296265, 0.39433833956718445, 0.4047241806983948, 0.40487736463546753, 0.395306259393692, 0.35388389229774475, 0.4141821265220642, 0.3878190815448761, 0.40146785974502563, 0.3607812225818634, 0.42003124952316284, 0.36781835556030273, 0.4019395709037781, 0.4063832759857178, 0.40765056014060974, 0.35358932614326477, 0.3856630325317383, 0.3571811318397522, 0.40113481879234314, 0.3867679536342621, 0.381091445684433, 0.3658832609653473, 0.3639684021472931, 0.3864218294620514, 0.40048930048942566, 0.41680869460105896, 0.32693132758140564, 0.38254183530807495, 0.3829517364501953, 0.40478089451789856, 0.37888893485069275, 0.387321799993515, 0.3875291049480438, 0.3538939356803894, 0.3909098505973816, 0.3872263729572296, 0.38510894775390625, 0.36245647072792053, 0.3746325671672821, 0.38296589255332947, 0.3871270716190338, 0.3788798153400421, 0.38520655035972595, 0.3798528015613556, 0.3944593071937561, 0.38830438256263733, 0.3603636920452118, 0.38289493322372437, 0.3635413944721222, 0.38682428002357483, 0.40535497665405273, 0.3861527740955353]
losse=[0.666441023349762, 0.443545401096344, 0.4402904808521271, 0.41923344135284424, 0.40610837936401367, 0.4354133903980255, 0.4100918173789978, 0.40635162591934204, 0.4041239321231842, 0.40100377798080444, 0.4028412401676178, 0.41881391406059265, 0.39728015661239624, 0.4003562331199646, 0.4011349081993103, 0.389141321182251, 0.4138961732387543, 0.40326550602912903, 0.40392830967903137, 0.38794854283332825, 0.40368518233299255, 0.41165873408317566, 0.38464340567588806, 0.3873163163661957, 0.41243594884872437, 0.3923010528087616, 0.40889692306518555, 0.39840763807296753, 0.3808088004589081, 0.40207257866859436, 0.4016839265823364, 0.3945044279098511, 0.3993243873119354, 0.38558343052864075, 0.3936128318309784, 0.38058415055274963, 0.3843187093734741, 0.3853977918624878, 0.37252265214920044, 0.3892264664173126, 0.377424031496048, 0.3721745014190674, 0.38715660572052, 0.3823164403438568, 0.3870171308517456, 0.3748817443847656, 0.4068020284175873, 0.3829105496406555, 0.37821173667907715, 0.38521310687065125, 0.37045013904571533, 0.37721696496009827, 0.37799766659736633, 0.38296109437942505, 0.3845213055610657, 0.38360336422920227, 0.389581561088562, 0.3840208649635315, 0.371692419052124, 0.3676907420158386, 0.3929951786994934, 0.37962183356285095, 0.3837434649467468, 0.3645118176937103, 0.3752654194831848, 0.37253111600875854, 0.40444105863571167, 0.39687687158584595, 0.38618138432502747, 0.38364624977111816, 0.38598179817199707, 0.36292433738708496, 0.39321988821029663, 0.377773642539978, 0.3815804123878479, 0.3753972351551056, 0.36458322405815125, 0.3598563075065613, 0.3723699450492859, 0.37792229652404785, 0.3774971663951874, 0.37759989500045776, 0.3747633695602417, 0.3724086284637451, 0.37157726287841797, 0.3746485710144043, 0.3860815763473511, 0.37445831298828125, 0.3731172978878021, 0.37790602445602417, 0.3680556118488312, 0.3752572536468506, 0.38026267290115356, 0.3717893660068512, 0.38101106882095337, 0.39427292346954346, 0.3680296540260315, 0.3675823509693146, 0.37638282775878906, 0.3839713931083679]
batch = [i*35 for i in range(len(losstn))]

In [14]:
plt.plot(batch, losstn, label = "train")
plt.plot(batch, losse, label = "eval")
plt.xlabel("Nb of batches (size_batch = 64)")
plt.ylabel("BCE loss")
plt.title("BCE loss evolution for the 1st epoch")
plt.legend()
plt.show()

plt.savefig("chart.png")

<Figure size 432x288 with 0 Axes>

## 6. Test and ROC Curves

In [10]:
CheXpertTrainer.test(model, dataLoaderTest, nnClassCount, checkpoint = "model1.pth.tar")

tensor([[0.0010, 0.1068, 0.3360, 0.6987, 0.0412, 0.2378, 0.4348, 0.1021, 0.5558,
         0.0440, 0.9719, 0.0139, 0.0142, 0.3197]], device='cuda:0')
tensor([[0.0010, 0.1068, 0.3360, 0.6987, 0.0412, 0.2378, 0.4348, 0.1021, 0.5558,
         0.0440, 0.9719, 0.0139, 0.0142, 0.3197]], device='cuda:0')
tensor([[0.0097, 0.0953, 0.1313, 0.6669, 0.0881, 0.4419, 0.3038, 0.2469, 0.2432,
         0.0780, 0.7633, 0.0370, 0.0326, 0.5039]], device='cuda:0')
tensor([[0.0010, 0.1068, 0.3360, 0.6987, 0.0412, 0.2378, 0.4348, 0.1021, 0.5558,
         0.0440, 0.9719, 0.0139, 0.0142, 0.3197],
        [0.0097, 0.0953, 0.1313, 0.6669, 0.0881, 0.4419, 0.3038, 0.2469, 0.2432,
         0.0780, 0.7633, 0.0370, 0.0326, 0.5039]], device='cuda:0')
tensor([[0.0038, 0.1264, 0.3808, 0.6322, 0.0141, 0.4408, 0.3117, 0.0651, 0.4780,
         0.0184, 0.7287, 0.0043, 0.0171, 0.7574]], device='cuda:0')
tensor([[0.0010, 0.1068, 0.3360, 0.6987, 0.0412, 0.2378, 0.4348, 0.1021, 0.5558,
         0.0440, 0.9719, 0.0139, 0.0142, 0.

tensor([[0.0863, 0.1099, 0.0526, 0.4212, 0.0407, 0.1384, 0.1304, 0.0778, 0.2559,
         0.0613, 0.2574, 0.0189, 0.0571, 0.6580]], device='cuda:0')
tensor([[1.0366e-03, 1.0684e-01, 3.3602e-01, 6.9868e-01, 4.1219e-02, 2.3779e-01,
         4.3479e-01, 1.0206e-01, 5.5578e-01, 4.3954e-02, 9.7186e-01, 1.3891e-02,
         1.4165e-02, 3.1968e-01],
        [9.7359e-03, 9.5320e-02, 1.3131e-01, 6.6694e-01, 8.8067e-02, 4.4195e-01,
         3.0384e-01, 2.4688e-01, 2.4318e-01, 7.8050e-02, 7.6326e-01, 3.6952e-02,
         3.2634e-02, 5.0389e-01],
        [3.7889e-03, 1.2638e-01, 3.8085e-01, 6.3221e-01, 1.4106e-02, 4.4081e-01,
         3.1174e-01, 6.5065e-02, 4.7799e-01, 1.8400e-02, 7.2867e-01, 4.2876e-03,
         1.7125e-02, 7.5744e-01],
        [4.8805e-03, 1.1999e-01, 3.2499e-01, 6.2918e-01, 1.3197e-02, 7.5653e-01,
         2.2448e-01, 7.6739e-02, 4.2170e-01, 3.2940e-02, 5.2024e-01, 4.0859e-03,
         2.6636e-02, 6.2460e-01],
        [2.6360e-03, 7.7465e-02, 1.5059e-01, 6.4168e-01, 9.9626e-03

tensor([[0.0042, 0.0999, 0.2428, 0.6615, 0.0384, 0.4269, 0.2766, 0.1508, 0.3449,
         0.0780, 0.6501, 0.0111, 0.0265, 0.4738]], device='cuda:0')
tensor([[1.0366e-03, 1.0684e-01, 3.3602e-01, 6.9868e-01, 4.1219e-02, 2.3779e-01,
         4.3479e-01, 1.0206e-01, 5.5578e-01, 4.3954e-02, 9.7186e-01, 1.3891e-02,
         1.4165e-02, 3.1968e-01],
        [9.7359e-03, 9.5320e-02, 1.3131e-01, 6.6694e-01, 8.8067e-02, 4.4195e-01,
         3.0384e-01, 2.4688e-01, 2.4318e-01, 7.8050e-02, 7.6326e-01, 3.6952e-02,
         3.2634e-02, 5.0389e-01],
        [3.7889e-03, 1.2638e-01, 3.8085e-01, 6.3221e-01, 1.4106e-02, 4.4081e-01,
         3.1174e-01, 6.5065e-02, 4.7799e-01, 1.8400e-02, 7.2867e-01, 4.2876e-03,
         1.7125e-02, 7.5744e-01],
        [4.8805e-03, 1.1999e-01, 3.2499e-01, 6.2918e-01, 1.3197e-02, 7.5653e-01,
         2.2448e-01, 7.6739e-02, 4.2170e-01, 3.2940e-02, 5.2024e-01, 4.0859e-03,
         2.6636e-02, 6.2460e-01],
        [2.6360e-03, 7.7465e-02, 1.5059e-01, 6.4168e-01, 9.9626e-03

tensor([[0.5985, 0.0952, 0.0057, 0.1305, 0.0728, 0.0082, 0.0347, 0.0245, 0.0762,
         0.1103, 0.0791, 0.0195, 0.0727, 0.7660]], device='cuda:0')
tensor([[1.0366e-03, 1.0684e-01, 3.3602e-01, 6.9868e-01, 4.1219e-02, 2.3779e-01,
         4.3479e-01, 1.0206e-01, 5.5578e-01, 4.3954e-02, 9.7186e-01, 1.3891e-02,
         1.4165e-02, 3.1968e-01],
        [9.7359e-03, 9.5320e-02, 1.3131e-01, 6.6694e-01, 8.8067e-02, 4.4195e-01,
         3.0384e-01, 2.4688e-01, 2.4318e-01, 7.8050e-02, 7.6326e-01, 3.6952e-02,
         3.2634e-02, 5.0389e-01],
        [3.7889e-03, 1.2638e-01, 3.8085e-01, 6.3221e-01, 1.4106e-02, 4.4081e-01,
         3.1174e-01, 6.5065e-02, 4.7799e-01, 1.8400e-02, 7.2867e-01, 4.2876e-03,
         1.7125e-02, 7.5744e-01],
        [4.8805e-03, 1.1999e-01, 3.2499e-01, 6.2918e-01, 1.3197e-02, 7.5653e-01,
         2.2448e-01, 7.6739e-02, 4.2170e-01, 3.2940e-02, 5.2024e-01, 4.0859e-03,
         2.6636e-02, 6.2460e-01],
        [2.6360e-03, 7.7465e-02, 1.5059e-01, 6.4168e-01, 9.9626e-03

tensor([[1.0366e-03, 1.0684e-01, 3.3602e-01, 6.9868e-01, 4.1219e-02, 2.3779e-01,
         4.3479e-01, 1.0206e-01, 5.5578e-01, 4.3954e-02, 9.7186e-01, 1.3891e-02,
         1.4165e-02, 3.1968e-01],
        [9.7359e-03, 9.5320e-02, 1.3131e-01, 6.6694e-01, 8.8067e-02, 4.4195e-01,
         3.0384e-01, 2.4688e-01, 2.4318e-01, 7.8050e-02, 7.6326e-01, 3.6952e-02,
         3.2634e-02, 5.0389e-01],
        [3.7889e-03, 1.2638e-01, 3.8085e-01, 6.3221e-01, 1.4106e-02, 4.4081e-01,
         3.1174e-01, 6.5065e-02, 4.7799e-01, 1.8400e-02, 7.2867e-01, 4.2876e-03,
         1.7125e-02, 7.5744e-01],
        [4.8805e-03, 1.1999e-01, 3.2499e-01, 6.2918e-01, 1.3197e-02, 7.5653e-01,
         2.2448e-01, 7.6739e-02, 4.2170e-01, 3.2940e-02, 5.2024e-01, 4.0859e-03,
         2.6636e-02, 6.2460e-01],
        [2.6360e-03, 7.7465e-02, 1.5059e-01, 6.4168e-01, 9.9626e-03, 6.9156e-01,
         2.8063e-01, 7.4511e-02, 3.6749e-01, 2.0442e-02, 8.8126e-01, 1.6455e-03,
         8.6766e-03, 8.9874e-01],
        [6.3796e-04,

...

tensor([[0.0154, 0.1083, 0.1356, 0.4645, 0.0768, 0.0498, 0.2864, 0.1176, 0.3744,
         0.0692, 0.8207, 0.0250, 0.0361, 0.1188]], device='cuda:0')
tensor([[0.0010, 0.1068, 0.3360,  ..., 0.0139, 0.0142, 0.3197],
        [0.0097, 0.0953, 0.1313,  ..., 0.0370, 0.0326, 0.5039],
        [0.0038, 0.1264, 0.3808,  ..., 0.0043, 0.0171, 0.7574],
        ...,
        [0.0019, 0.0903, 0.2961,  ..., 0.0027, 0.0113, 0.7433],
        [0.1307, 0.1618, 0.1612,  ..., 0.0099, 0.1082, 0.3283],
        [0.0154, 0.1083, 0.1356,  ..., 0.0250, 0.0361, 0.1188]],
       device='cuda:0')
tensor([[0.0036, 0.1043, 0.1311, 0.7575, 0.1411, 0.5569, 0.2529, 0.4010, 0.0959,
         0.1439, 0.4321, 0.0708, 0.0263, 0.4935]], device='cuda:0')
tensor([[0.0010, 0.1068, 0.3360,  ..., 0.0139, 0.0142, 0.3197],
        [0.0097, 0.0953, 0.1313,  ..., 0.0370, 0.0326, 0.5039],
        [0.0038, 0.1264, 0.3808,  ..., 0.0043, 0.0171, 0.7574],
        ...,
        [0.1307, 0.1618, 0.1612,  ..., 0.0099, 0.1082, 0.3283],
        [0.

tensor([[0.0395, 0.1060, 0.1956, 0.4327, 0.0582, 0.1075, 0.1839, 0.1165, 0.3065,
         0.0717, 0.3983, 0.0310, 0.0637, 0.1291]], device='cuda:0')
tensor([[0.0010, 0.1068, 0.3360,  ..., 0.0139, 0.0142, 0.3197],
        [0.0097, 0.0953, 0.1313,  ..., 0.0370, 0.0326, 0.5039],
        [0.0038, 0.1264, 0.3808,  ..., 0.0043, 0.0171, 0.7574],
        ...,
        [0.0174, 0.1019, 0.1198,  ..., 0.0170, 0.0353, 0.7502],
        [0.2647, 0.1096, 0.0897,  ..., 0.0333, 0.0848, 0.5875],
        [0.0395, 0.1060, 0.1956,  ..., 0.0310, 0.0637, 0.1291]],
       device='cuda:0')
tensor([[0.0354, 0.0975, 0.0553, 0.5392, 0.0180, 0.2637, 0.2417, 0.0570, 0.3860,
         0.0840, 0.4297, 0.0067, 0.0568, 0.9019]], device='cuda:0')
tensor([[0.0010, 0.1068, 0.3360,  ..., 0.0139, 0.0142, 0.3197],
        [0.0097, 0.0953, 0.1313,  ..., 0.0370, 0.0326, 0.5039],
        [0.0038, 0.1264, 0.3808,  ..., 0.0043, 0.0171, 0.7574],
        ...,
        [0.2647, 0.1096, 0.0897,  ..., 0.0333, 0.0848, 0.5875],
        [0.

tensor([[0.0071, 0.0869, 0.1612, 0.5427, 0.0540, 0.1570, 0.2524, 0.2257, 0.4226,
         0.0203, 0.5044, 0.0184, 0.0313, 0.0335]], device='cuda:0')
tensor([[0.0010, 0.1068, 0.3360,  ..., 0.0139, 0.0142, 0.3197],
        [0.0097, 0.0953, 0.1313,  ..., 0.0370, 0.0326, 0.5039],
        [0.0038, 0.1264, 0.3808,  ..., 0.0043, 0.0171, 0.7574],
        ...,
        [0.0042, 0.1238, 0.1181,  ..., 0.0134, 0.0270, 0.8712],
        [0.2065, 0.1145, 0.0755,  ..., 0.0441, 0.0532, 0.0912],
        [0.0071, 0.0869, 0.1612,  ..., 0.0184, 0.0313, 0.0335]],
       device='cuda:0')
tensor([[0.0091, 0.1567, 0.6091, 0.5592, 0.0196, 0.6520, 0.1985, 0.0634, 0.3416,
         0.0313, 0.5676, 0.0102, 0.0194, 0.7476]], device='cuda:0')
tensor([[0.0010, 0.1068, 0.3360,  ..., 0.0139, 0.0142, 0.3197],
        [0.0097, 0.0953, 0.1313,  ..., 0.0370, 0.0326, 0.5039],
        [0.0038, 0.1264, 0.3808,  ..., 0.0043, 0.0171, 0.7574],
        ...,
        [0.2065, 0.1145, 0.0755,  ..., 0.0441, 0.0532, 0.0912],
        [0.

tensor([[0.0915, 0.1483, 0.2624, 0.3136, 0.0427, 0.1207, 0.1011, 0.0520, 0.2826,
         0.0196, 0.1596, 0.0160, 0.0478, 0.5314]], device='cuda:0')
tensor([[0.0010, 0.1068, 0.3360,  ..., 0.0139, 0.0142, 0.3197],
        [0.0097, 0.0953, 0.1313,  ..., 0.0370, 0.0326, 0.5039],
        [0.0038, 0.1264, 0.3808,  ..., 0.0043, 0.0171, 0.7574],
        ...,
        [0.0030, 0.1314, 0.4993,  ..., 0.0081, 0.0167, 0.3946],
        [0.0467, 0.1446, 0.0466,  ..., 0.0288, 0.0330, 0.7940],
        [0.0915, 0.1483, 0.2624,  ..., 0.0160, 0.0478, 0.5314]],
       device='cuda:0')
tensor([[0.0041, 0.1356, 0.3963, 0.6627, 0.0047, 0.5968, 0.2794, 0.0282, 0.3998,
         0.0171, 0.7949, 0.0023, 0.0083, 0.9628]], device='cuda:0')
tensor([[0.0010, 0.1068, 0.3360,  ..., 0.0139, 0.0142, 0.3197],
        [0.0097, 0.0953, 0.1313,  ..., 0.0370, 0.0326, 0.5039],
        [0.0038, 0.1264, 0.3808,  ..., 0.0043, 0.0171, 0.7574],
        ...,
        [0.0467, 0.1446, 0.0466,  ..., 0.0288, 0.0330, 0.7940],
        [0.

tensor([[0.5697, 0.0910, 0.0225, 0.1294, 0.0610, 0.0180, 0.0316, 0.0304, 0.0408,
         0.0221, 0.0802, 0.0273, 0.0375, 0.8803]], device='cuda:0')
tensor([[0.0010, 0.1068, 0.3360,  ..., 0.0139, 0.0142, 0.3197],
        [0.0097, 0.0953, 0.1313,  ..., 0.0370, 0.0326, 0.5039],
        [0.0038, 0.1264, 0.3808,  ..., 0.0043, 0.0171, 0.7574],
        ...,
        [0.0042, 0.0893, 0.3637,  ..., 0.0269, 0.0199, 0.0699],
        [0.0037, 0.0990, 0.2767,  ..., 0.0287, 0.0336, 0.1549],
        [0.5697, 0.0910, 0.0225,  ..., 0.0273, 0.0375, 0.8803]],
       device='cuda:0')
tensor([[0.0208, 0.1399, 0.0848, 0.5633, 0.0559, 0.0501, 0.2500, 0.0816, 0.4854,
         0.1967, 0.7229, 0.0334, 0.0498, 0.3130]], device='cuda:0')
tensor([[0.0010, 0.1068, 0.3360,  ..., 0.0139, 0.0142, 0.3197],
        [0.0097, 0.0953, 0.1313,  ..., 0.0370, 0.0326, 0.5039],
        [0.0038, 0.1264, 0.3808,  ..., 0.0043, 0.0171, 0.7574],
        ...,
        [0.0037, 0.0990, 0.2767,  ..., 0.0287, 0.0336, 0.1549],
        [0.

## 5. Heatmap generator

First we define a transform model to resize all images and normalize them.

In [ ]:
class HeatmapGenerator ():
    
    #---- Initialize heatmap generator
    #---- pathModel - path to the trained densenet model
    #---- nnArchitecture - architecture name DENSE-NET121, DENSE-NET169, DENSE-NET201
    #---- nnClassCount - class count, 14 for chxray-14

 
    def __init__ (self, pathModel, nnArchitecture, nnClassCount, transCrop):
       
        #---- Initialize the network
        if nnArchitecture == 'DENSE-NET-121': model = DenseNet121(nnClassCount).cuda()
        
        if use_gpu:
            model = torch.nn.DataParallel(model).cuda()
        else:
            model = torch.nn.DataParallel(model)
        
        modelCheckpoint = torch.load(pathModel)
        model.load_state_dict(modelCheckpoint['state_dict'])

        self.model = model
        self.model.eval()
        
        #---- Initialize the weights
        self.weights = list(self.model.module.densenet121.features.parameters())[-2]

        #---- Initialize the image transform
        normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transformList = []
        transformList.append(transforms.Resize((transCrop, transCrop)))
        transformList.append(transforms.ToTensor())
        transformList.append(normalize)  
        self.transformSequence = transforms.Compose(transformList)
    
    #--------------------------------------------------------------------------------
     
    def generate (self, pathImageFile, pathOutputFile, transCrop):
        
        #---- Load image, transform, convert 
        with torch.no_grad():
 
            imageData = Image.open(pathImageFile).convert('RGB')
            imageData = self.transformSequence(imageData)
            imageData = imageData.unsqueeze_(0)
            if use_gpu:
                imageData = imageData.cuda()
            l = self.model(imageData)
            output = self.model.module.densenet121.features(imageData)
            label = class_names[torch.max(l,1)[1]]
            #---- Generate heatmap
            heatmap = None
            for i in range (0, len(self.weights)):
                map = output[0,i,:,:]
                if i == 0: heatmap = self.weights[i] * map
                else: heatmap += self.weights[i] * map
                npHeatmap = heatmap.cpu().data.numpy()

        #---- Blend original and heatmap 
                
        imgOriginal = cv2.imread(pathImageFile, 1)
        imgOriginal = cv2.resize(imgOriginal, (transCrop, transCrop))
        
        cam = npHeatmap / np.max(npHeatmap)
        cam = cv2.resize(cam, (transCrop, transCrop))
        heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
        
        img = cv2.addWeighted(imgOriginal,1,heatmap,0.35,0)            
        cv2.imwrite(pathOutputFile, img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.title(label)
        plt.imshow(img)
        plt.plot()


In [ ]:
pathInputImage = '../CheXpert-v1.0-small/valid/patient64698/study1/view1_frontal.jpg'
pathOutputImage = 'heatmap_patient64698_1.png'
pathModel = "m-07032019-175037.pth.tar"


h = HeatmapGenerator(pathModel, nnArchitecture, nnClassCount, imgtransCrop)

h.generate(pathInputImage, pathOutputImage, imgtransCrop)